In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.datasets.qm9 import QM9
import torch_geometric.datasets.qm9 as qm9
from torch_geometric.data import DataLoader
import torch_geometric.nn as tgnn
from torch_scatter import scatter
import tqdm
import numpy as np

In [2]:
# [0] Reports MAE in eV / Chemical Accuracy of the target variable U0. 
# The chemical accuracy of U0 is 0.043 see [1, Table 5].

# Reproduced table [0]
# MXMNet: 0.00590/0.043 = 0.13720930232558143
# HMGNN:  0.00592/0.043 = 0.13767441860465118
# MPNN:   0.01935/0.043 = 0.45
# KRR:    0.0251 /0.043 = 0.5837209302325582
# [0] https://paperswithcode.com/sota/formation-energy-on-qm9
# [1] Neural Message Passing for Quantum Chemistry, https://arxiv.org/pdf/1704.01212v2.pdf
# MXMNet https://arxiv.org/pdf/2011.07457v1.pdf
# HMGNN https://arxiv.org/pdf/2009.12710v1.pdf
# MPNN https://arxiv.org/pdf/1704.01212v2.pdf
# KRR HDAD kernel ridge regression https://arxiv.org/pdf/1702.05532.pdf
# HDAD means HDAD (Histogram of distances, anglesand dihedral angles)

# [2] Reports the average value of MAE / Chemical Accuracy of over all targets
# [2] https://paperswithcode.com/sota/drug-discovery-on-qm9
target_dict = {0: 'mu, D, Dipole moment', 
               1: 'alpha, {a_0}^3, Isotropic polarizability', 
               2: 'epsilon_{HOMO}, eV, Highest occupied molecular orbital energy',
               3: 'epsilon_{LUMO}, eV, Lowest unoccupied molecular orbital energy',
               4: 'Delta, eV, Gap between HOMO and LUMO',
               5: '< R^2 >, {a_0}^2, Electronic spatial extent',
               6: 'ZPVE, eV, Zero point vibrational energy', 
               7: 'U_0, eV, Internal energy at 0K',
               8: 'U, eV, Internal energy at 298.15K', 
               9: 'H, eV, Enthalpy at 298.15K',
               10: 'G, eV, Free energy at 298.15K',  
               11: 'c_{v}, cal\(mol K), Heat capacity at 298.15K'}

chemical_accuracy = {idx:0.043 for idx in range(12)}
chemical_accuracy[0] = 0.1
chemical_accuracy[1] = 0.1
chemical_accuracy[5] = 1.2
chemical_accuracy[6] = 0.0012
chemical_accuracy[11] = 0.050

In [3]:
dataset = QM9('../datasets/qm9_geometric/')

/home/chrisw/.cache/pypoetry/virtualenvs/graph-transformer-9jPERXQ--py3.8/lib/python3.8/site-packages/torch_geometric/data/dataset.py:153: UserWarning: The `pre_filter` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-fitering technique, make sure to delete '{self.processed_dir}' first
  warnings.warn(


In [4]:
dataset = dataset.shuffle()
train_dataset = dataset[:110000]
valid_dataset = dataset[110000:120000]
test_dataset = dataset[120000:]
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=128)
test_loader = DataLoader(test_dataset, batch_size=32)

In [5]:
ngpu=1
device = torch.device('cuda:0' if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

# Model

In [6]:
#lr = 0.0003 did not diverge
#lr = 0.0003
lr = 0.0003
n_epochs = 10000
patience = 5
factor = 0.8
target_idx = 7

dataset.atomref(target_idx)

In [7]:
target_vec = []

In [8]:
# based on https://schnetpack.readthedocs.io/en/stable/tutorials/tutorial_02_qm9.html
# and https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/models/schnet.html#SchNet
for data in train_loader:
    data = data.to(device)
    atomU0s = torch.tensor(qm9.atomrefs[target_idx], device=device)[torch.argmax(data.x[:, :5], axis=1)]
    target_modular = scatter(atomU0s, data.batch, dim=-1, reduce='sum')
    target_vec += [(data.y[:, target_idx] - target_modular).detach().cpu().numpy()]
target_vec = np.concatenate(target_vec, axis=0)

In [9]:
target_mean = np.mean(target_vec)
target_std = np.std(target_vec)

In [10]:
model = tgnn.SchNet(hidden_channels=128, num_filters=128, num_interactions=6, num_gaussians=50, cutoff=10.0,
                    readout='add', atomref=dataset.atomref(target_idx), mean=target_mean, std=target_std).to(device)
loss = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=lr)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', verbose=True, patience=patience, factor=factor)

# Training

In [ ]:
loss_hist = []
try:
    # For each epoch
    for epoch in range(n_epochs):
        # For each batch in the dataloader
        pbar = tqdm.tqdm(enumerate(train_loader, 0))
        epoch_loss = 0
        for i, data in pbar:
            model.zero_grad()
            data.to(device)
            target = data.y[:, target_idx]
            prediction = model(data.z, data.pos, data.batch)
            output = loss(prediction.view(-1), target)
            mae = (prediction.view(-1) - target).abs().mean()
            epoch_loss = (epoch_loss*i + mae.item())/(i+1)
            
            pbar.set_description('Epoch %d: MAE/CA %2.6f'%(epoch+1, epoch_loss/chemical_accuracy[target_idx]))
            output.backward()
            optimizer.step()

        lr_scheduler.step(epoch_loss)
        loss_hist += [epoch_loss]
        if epoch_loss/chemical_accuracy[target_idx] < 0.45:
            break

except KeyboardInterrupt:
    print('keyboard interrupt caught')

Epoch 1: MAE/CA 509.808559: : 860it [00:30, 28.29it/s] 
Epoch 2: MAE/CA 120.301568: : 860it [00:30, 28.66it/s]
Epoch 3: MAE/CA 103.046359: : 860it [00:30, 28.41it/s]
Epoch 4: MAE/CA 98.339732: : 860it [00:29, 28.73it/s]
Epoch 5: MAE/CA 76.110850: : 860it [00:30, 28.61it/s]
Epoch 6: MAE/CA 84.113742: : 860it [00:29, 28.67it/s]
Epoch 7: MAE/CA 77.365542: : 860it [00:29, 28.84it/s]
Epoch 8: MAE/CA 72.991522: : 860it [00:29, 28.98it/s] 
Epoch 9: MAE/CA 66.861041: : 860it [00:29, 28.76it/s] 
Epoch 10: MAE/CA 65.695961: : 860it [00:30, 28.66it/s]
Epoch 11: MAE/CA 55.602410: : 860it [00:30, 28.61it/s]
Epoch 12: MAE/CA 56.998222: : 860it [00:30, 28.64it/s]
Epoch 13: MAE/CA 37.282655: : 860it [00:30, 28.47it/s]
Epoch 14: MAE/CA 40.799564: : 860it [00:30, 28.18it/s]
Epoch 15: MAE/CA 40.948851: : 860it [00:30, 28.49it/s]
Epoch 16: MAE/CA 39.664054: : 860it [00:30, 28.62it/s]
Epoch 17: MAE/CA 31.738957: : 860it [00:30, 28.53it/s]
Epoch 18: MAE/CA 38.046365: : 860it [00:29, 29.03it/s]
Epoch 19: MAE

Epoch    33: reducing learning rate of group 0 to 2.4000e-04.


Epoch 34: MAE/CA 11.296434: : 860it [00:29, 29.26it/s]
Epoch 35: MAE/CA 16.875772: : 860it [00:29, 29.09it/s]
Epoch 36: MAE/CA 17.765804: : 860it [00:28, 30.08it/s]
Epoch 37: MAE/CA 17.012156: : 860it [00:29, 29.17it/s]
Epoch 38: MAE/CA 14.147567: : 860it [00:29, 29.56it/s]
Epoch 39: MAE/CA 16.913523: : 860it [00:29, 29.35it/s]
Epoch 40: MAE/CA 11.789189: : 3it [00:00, 27.18it/s]

Epoch    39: reducing learning rate of group 0 to 1.9200e-04.


Epoch 40: MAE/CA 6.494423: : 860it [00:29, 28.94it/s]
Epoch 41: MAE/CA 13.140620: : 860it [00:29, 29.38it/s]
Epoch 42: MAE/CA 13.355902: : 860it [00:29, 29.44it/s]
Epoch 43: MAE/CA 12.719528: : 860it [00:29, 29.14it/s]
Epoch 44: MAE/CA 12.330629: : 860it [00:28, 30.23it/s]
Epoch 45: MAE/CA 10.767876: : 860it [00:28, 29.85it/s]
Epoch 46: MAE/CA 11.386997: : 860it [00:29, 29.17it/s]
Epoch 47: MAE/CA 4.558401: : 3it [00:00, 28.66it/s]

...

Epoch 315: MAE/CA 1.078081: : 860it [00:28, 30.01it/s]
Epoch 316: MAE/CA 1.079144: : 860it [00:29, 29.55it/s]
Epoch 317: MAE/CA 1.078369: : 860it [00:28, 29.76it/s]
Epoch 318: MAE/CA 1.077174: : 860it [00:29, 29.36it/s]
Epoch 319: MAE/CA 1.077046: : 860it [00:27, 31.35it/s]
Epoch 320: MAE/CA 1.077547: : 860it [00:28, 30.53it/s]
Epoch 321: MAE/CA 1.079698: : 860it [00:28, 29.73it/s]
Epoch 322: MAE/CA 1.077005: : 860it [00:29, 29.62it/s]
Epoch 323: MAE/CA 1.075133: : 860it [00:29, 29.27it/s]
Epoch 324: MAE/CA 1.077031: : 860it [00:29, 29.14it/s]
Epoch 325: MAE/CA 1.078752: : 860it [00:29, 29.46it/s]
Epoch 326: MAE/CA 1.077888: : 860it [00:29, 29.24it/s]
Epoch 327: MAE/CA 1.079703: : 860it [00:29, 29.47it/s]
Epoch 328: MAE/CA 1.076867: : 860it [00:29, 29.35it/s]
Epoch 329: MAE/CA 1.075674: : 860it [00:29, 29.52it/s]
Epoch 330: MAE/CA 1.033472: : 3it [00:00, 29.38it/s]

Epoch   329: reducing learning rate of group 0 to 4.6423e-07.


Epoch 330: MAE/CA 1.072877: : 860it [00:29, 29.26it/s]
Epoch 331: MAE/CA 1.077143: : 860it [00:29, 29.27it/s]
Epoch 332: MAE/CA 1.070585: : 860it [00:29, 29.32it/s]
Epoch 333: MAE/CA 1.073223: : 860it [00:29, 29.54it/s]
Epoch 334: MAE/CA 1.071211: : 860it [00:29, 29.64it/s]
Epoch 335: MAE/CA 1.071266: : 860it [00:29, 29.49it/s]
Epoch 336: MAE/CA 1.073477: : 860it [00:29, 29.17it/s]
Epoch 337: MAE/CA 1.075716: : 860it [00:29, 29.27it/s]
Epoch 338: MAE/CA 1.070577: : 860it [00:29, 29.59it/s]
Epoch 339: MAE/CA 1.107469: : 3it [00:00, 27.37it/s]

Epoch   338: reducing learning rate of group 0 to 3.7138e-07.


Epoch 339: MAE/CA 1.069700: : 860it [00:29, 29.09it/s]
Epoch 340: MAE/CA 1.067161: : 860it [00:29, 29.51it/s]
Epoch 341: MAE/CA 1.066361: : 837it [00:28, 29.41it/s]

In [ ]:
model = model.to('cpu')
pbar = tqdm.tqdm(enumerate(test_loader, 0))
epoch_loss = 0
maes = []
for i, data in pbar:
    data.to('cpu')
    prediction = model(data.z, data.x, data.batch)
    mae = (denormalize(prediction.view(-1)) - data.y[:, target_idx]).abs()
    maes += [mae]
maes = torch.cat(maes, dim=0)
mae = maes.mean().item()
print(mae, mae/chemical_accuracy[target_idx])
